# 1 - Centralidade Closeness

In [14]:
import netpixi
import graph_tool_extras as gte

from graph_tool import centrality

## Motivação

Na rede abaixo, os vértices são famílias da elite da Florença no século XV e as arestas representam relações de casamento ou negócio [[1](#medici), [2](#duality)].

In [ ]:
netpixi.render('../../../Datasets/florentine-families-clean.net.gz');

A importância histórica dessas famílias é tão significativa que foi retratada em uma [série de televisão](https://www.imdb.com/title/tt5057130/). Para historiadores, é particularmente importante entender as causas e consequências da influência política de cada uma delas. As relações de casamento e negócio eram fundamentais para essas influências, pois informações podiam passar de uma família para outra através dessas relações. Faz sentido, portanto, extrair métricas dessa rede para entender o poder que cada família detinha sobre essas informações.

Este tutorial, em particular, busca responder à seguinte pergunta: *como calcular o quão rapidamente uma família podia receber informações sobre outras famílias?*

## Intuição

Podemos observar na rede acima que, em alguns casos, uma informação não precisava de muitos passos para chegar de uma família a outra. A família Medici, por exemplo, não estava conectada diretamente à família Ginori, mas, para receber uma informação sobre essa família, bastava que ela passasse dos Ginori para os Albizzi e dos Albizzi para os Medici.

Por outro lado, em outros casos, uma informação precisava de muitos passos para chegar. Para que a família Pazzi recebesse uma informação sobre a família Peruzzi, por exemplo, era necessário que ela passasse por no mínimo outras quatro famílias.

A partir dessas observações, vamos considerar a seguinte ideia: *para cada família, calcular o inverso do número médio de passos necessários para chegar dessa família a qualquer outra.* Por que o inverso? Porque o número de passos representa *distância,* mas queremos uma métrica que representa *proximidade.*

## Definição

Seja $n$ o número de vértices e seja $d(u, v)$ a distância de um vértice $u$ a outro vértice $v$.

A **centralidade closeness** de $u$ é dada por

$$\sum_{v \neq u} \frac{n - 1}{d(u, v)}.$$

Ou seja, essa centralidade é o inverso da média de $d(u, v)$ para todo $v \neq u$. Por ser o inverso, está naturalmente entre $0$ e $1$.

## Cálculo

Para calcular a centralidade closeness, podemos chamar a função `closeness` da [graph-tool](https://graph-tool.skewed.de/).

Essa função devolve uma propriedade de vértice. Podemos adicionar essa propriedade ao grafo passando ela como segundo parâmetro do método `add_vp`.

In [30]:
g = gte.load('movies_reposicionado4.net.gz')

# O parâmetro harmonic=True é necessário para garantir
# que a fórmula seja exatamente a definida acima.
c = centrality.closeness(g, harmonic=True)

g.add_vp('closeness', c)

In [31]:
import pandas as pd
dicio = {"id": g.vp["id"], "closeness": c}
df = pd.DataFrame(dicio)
df = df.sort_values(by="closeness", ascending=False)
df.head()

,id,closeness
7,Drama,0.619515
4,Comedy,0.533009
1154,1138,0.451630
353,337,0.446081
21,4,0.445934


In [32]:
df["id"].value_counts()

id
488       1
Drama     1
811       1
1585      1
1594      1
         ..
74        1
655       1
312       1
Action    1
842       1
Name: count, Length: 1698, dtype: int64

## Visualização

Lembre que, para visualizar a rede modificada, precisamos salvar ela em um novo arquivo.

In [33]:
gte.save(g, 'closeness_twomode2.net.gz')

Como a rede não é muito grande, vamos desconsiderar o tutorial de visualização em larga escala e rodar uma segunda visualização.

In [34]:
r = netpixi.render('closeness_twomode2.net.gz')

In [35]:
r.vertex_set_key("name")

Nessa visualização, podemos chamar o método `vertex_scale` para usar o closeness como tamanho dos vértices.

Esse método recebe três parâmetros: o primeiro é o nome da propriedade, o segundo é o tamanho mínimo e o terceiro é o tamanho máximo.

In [36]:
r.vertex_scale('closeness', 10, 30)

## Referências

[<a name="medici">1</a>] J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400-1434.* American Journal of Sociology, 98(6), 1993.

[<a name="duality">2</a>] R. L. Breiger e P. E. Pattison. *Cumulated social roles: the duality of persons and their algebras.* Social Networks, 8(3), 1986.

<div class="alert alert-block alert-info">
   <strong>DICA:</strong> Você pode clicar na aba
   
   à esquerda para navegar por seção.
</div>